In [157]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [158]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)


In [159]:
# Determine the number of unique values in each column.

application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [160]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [161]:
# Choose a cutoff value
cutoff_value = 67

# Bin application types with values below cutoff into 'Other'
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].apply(lambda x: x if application_df['APPLICATION_TYPE'].value_counts()[x] >= cutoff_value else 'Other')

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [162]:
# Look at CLASSIFICATION value counts for binning

application_df['CLASSIFICATION'].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

### Filters the application dataframe to include only the top performing classifications based on success rate.


In [163]:
# Calculate the success rate for each CLASSIFICATION
success_rates = application_df.groupby('CLASSIFICATION')['IS_SUCCESSFUL'].mean()

# Sort the classifications by success rate
sorted_success_rates = success_rates.sort_values(ascending=False)

# Decide on a threshold or number of top categories you want to keep
# For example, keep top 25% performing classifications
top_classifications = sorted_success_rates.head(int(len(sorted_success_rates) * 0.50)).index.tolist()

# Filter the application_df to only include the top classifications
top_classification_df = application_df[application_df['CLASSIFICATION'].isin(top_classifications)]

# Check to see most succesful classification type from the top 25%
most_common_classification = top_classification_df['CLASSIFICATION'].value_counts()
most_common_classification



CLASSIFICATION
C7000    777
C7100     75
C1230     36
C7200     32
C1240     30
C7120     18
C6000     15
C1800     15
C1278     10
C1238     10
C1235      9
C7210      7
C4100      6
C1257      5
C1246      2
C1234      2
C1256      2
C3700      1
C1900      1
C1248      1
C1370      1
C4200      1
C2570      1
C1580      1
C1570      1
C2500      1
C1283      1
C1728      1
C2170      1
C4120      1
C8210      1
C4500      1
C1245      1
C2561      1
C2150      1
Name: count, dtype: int64

In [164]:
# Choose a cutoff value
most_common_classification = top_classification_df['CLASSIFICATION'].value_counts().index.tolist()

# Create a new column 'Top_Classification' in application_df
application_df['TOP_CLASSIFICATION'] = application_df['CLASSIFICATION'].apply(lambda x: True if x in most_common_classification else False)

# Check the updated dataframe by looking at the 50 top companies within the top classification
top_classification_company = application_df[application_df['TOP_CLASSIFICATION'] == True].head(50)
top_classification_company





,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,TOP_CLASSIFICATION
22,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,318296,0,True
44,T3,Independent,C7200,Preservation,Trust,1,0,N,5000,1,True
69,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,6501,1,True
71,T3,Independent,C7000,Preservation,Trust,1,100000-499999,N,5000,1,True
78,T3,Independent,C7000,Preservation,Trust,1,0,N,5000,1,True
82,T3,Independent,C7000,Preservation,Trust,1,0,N,5000,0,True
84,T3,Independent,C7100,Preservation,Trust,1,25000-99999,N,5000,1,True
121,T3,Independent,C6000,Preservation,Trust,1,0,N,5000,1,True
141,T3,Independent,C1238,Preservation,Trust,1,0,N,5000,1,True
160,T3,Independent,C7000,Preservation,Trust,1,25000-99999,N,452970,1,True


In [165]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True) 
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,TOP_CLASSIFICATION,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,False,True,False,False,False,...,True,False,True,False,False,False,False,False,False,False
4,1,142590,1,False,False,False,True,False,False,False,...,True,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
34295,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
34296,1,5000,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34297,1,5000,1,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [166]:
# Drop columns that are less likely to contribute meaningfully
columns_to_drop = ['STATUS']  # Dropping one of the binary opposites for simplification

# However, since these decisions require more detailed analysis, they are not included in the direct drop list here.

application_df.drop(columns=columns_to_drop, axis=1, inplace=True)

# Continue with your model preparation and evaluation


In [167]:
# Split the preprocessed data into features (X) and target (y) arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # added test_size for clarity

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler with the training data
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train, and Evaluate the Model

In [168]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

nn = Sequential()

# Input layer and first hidden layer with dropout
nn.add(Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))  # Adjusted activation to 'relu'
nn.add(Dropout(0.2))  # Keeping dropout at 20%

# Second hidden layer with dropout
nn.add(Dense(units=40, activation='relu'))  # Adjusted the number of units and activation to 'relu'
nn.add(Dropout(0.2))  # Keeping dropout at 20%

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 80)                8240      
                                                                 
 dropout_25 (Dropout)        (None, 80)                0         


                                                                 
 dense_41 (Dense)            (None, 40)                3240      
                                                                 
 dropout_26 (Dropout)        (None, 40)                0         
                                                                 
 dense_42 (Dense)            (None, 1)                 41        
                                                                 
Total params: 11521 (45.00 KB)
Trainable params: 11521 (45.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [169]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callback to save the model's weights every 5 epochs
checkpoint_path_periodic = "checkpoints/weights_every_5_epochs.{epoch:02d}.hdf5"
cp_callback_periodic = ModelCheckpoint(
    filepath=checkpoint_path_periodic,
    verbose=1,
    save_weights_only=True,
    period=5  # Saves every 5 epochs
)

# Callback to save the best model according to the validation loss
checkpoint_path_best = "checkpoints/weights_best.hdf5"
cp_callback_best = ModelCheckpoint(
    filepath=checkpoint_path_best,
    verbose=1,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True  # Saves only the best model
)

# EarlyStopping callback
es_callback = EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=20,
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity
)

callbacks=[cp_callback_periodic, cp_callback_best, es_callback]


# Train the model with the callbacks list
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=40, validation_split=0.15, callbacks=callbacks, verbose=1)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/100
578/584 [============================>.] - ETA: 0s - loss: 0.6036 - accuracy: 0.7021
Epoch 1: val_loss improved from inf to 0.55603, saving model to checkpoints\weights_best.hdf5
584/584 [==============================] - 1s 1ms/step - loss: 0.6038 - accuracy: 0.7020 - val_loss: 0.5560 - val_accuracy: 0.7371
Epoch 2/100
547/584 [===========================>..] - ETA: 0s - loss: 0.5708 - accuracy: 0.7224
Epoch 2: val_loss improved from 0.55603 to 0.55422, saving model to checkpoints\weights_best.hdf5
584/584 [==============================] - 1s 993us/step - loss: 0.5698 - accuracy: 0.7241 - val_loss: 0.5542 - val_accuracy: 0.7369
Epoch 3/100
532/584 [==========================>...] - ETA: 0s - loss: 0.5619 - accuracy: 0.7275
Epoch 3: val_loss improved from 0.55422 to 0.55308, saving model to checkpoints\weights_best.hdf5
584/584 [==============================] - 1s 990us/step - loss: 0.5630 - accuracy: 0.7257 - val_loss: 0.5531 - val_accuracy: 0.7388
Epoch 4/100
539/584 [=

In [170]:
# Export our model to HDF5 file
trained_path = "./trained_model/trained_attrition.h5"
nn.save(trained_path)


# Optimize the Model